In [1]:
# Part 1: Preproccesing
import pandas as pd
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk import PorterStemmer
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from torch.autograd import Variable

local_file="emails-1.csv"
df = pd.read_csv(local_file,sep=',',header=0,engine='python')
# Use pd.read_csv to read the csv
print(df)

C:\Users\Jeff\anaconda3\lib\site-packages\scipy\__init__.py:138: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.4)
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion} is required for this version of "


                                                   text  spam
0     Subject: naturally irresistible your corporate...     1
1     Subject: the stock trading gunslinger  fanny i...     1
2     Subject: unbelievable new homes made easy  im ...     1
3     Subject: 4 color printing special  request add...     1
4     Subject: do not have money , get software cds ...     1
...                                                 ...   ...
5723  Subject: re : research and development charges...     0
5724  Subject: re : receipts from visit  jim ,  than...     0
5725  Subject: re : enron case study update  wow ! a...     0
5726  Subject: re : interest  david ,  please , call...     0
5727  Subject: news : aurora 5 . 2 update  aurora ve...     0

[5728 rows x 2 columns]


In [2]:
def stem(text):
    ps=PorterStemmer()
    return [ps.stem(word) for word in text]

#tokenize the words 
token=lambda x:word_tokenize(x)

# Transform elements
df['text']=df['text'].apply(token)
df['text']=df['text'].apply(lambda x:stem(x)) # 
df['text']=df['text'].apply(lambda x:' '.join(x))
print(df.head())

                                                text  spam
0  subject : natur irresist your corpor ident lt ...     1
1  subject : the stock trade gunsling fanni is me...     1
2  subject : unbeliev new home made easi im want ...     1
3  subject : 4 color print special request addit ...     1
4  subject : do not have money , get softwar cd f...     1


In [3]:
maxfea=5000
cv = CountVectorizer(max_features = maxfea, stop_words = 'english')
fea = cv.fit_transform(df['text']).toarray()
text_train, text_test, spam_train, spam_test= train_test_split(fea, np.array(df['spam']))

In [4]:
import torch
import torch.nn as nn

In [5]:
# Part 2: Logistic Regression
class LogisticRegression(nn.Module):
    def __init__(self):
        super(LogisticRegression, self).__init__()
        self.l1 = nn.Linear(5000, 1000)
        self.l2 = nn.Linear(1000, 100)
        self.l3 = nn.Linear(100, 10)
        self.l4 = nn.Linear(10, 2)
    def forward(self, x):
        x = torch.nn.functional.relu(self.l1(x))
        x = torch.nn.functional.relu(self.l2(x))
        x = torch.nn.functional.relu(self.l3(x))
        x = self.l4(x)
        return x

In [6]:
text_train = Variable(torch.from_numpy(text_train)).float()
spam_train = Variable(torch.from_numpy(spam_train)).long()
text_test = Variable(torch.from_numpy(text_test)).float()
spam_test = Variable(torch.from_numpy(spam_test)).long()

In [7]:
import gc
gc.collect()

25

In [8]:
def LR(text_train,spam_train):
    LR = LogisticRegression()
    criterion = nn.CrossEntropyLoss()
    # Adam is an optimization algorithm that combines the advantages of two other extensions of stochastic gradient descent:
    # Adaptive Gradient Algorithm (AdaGrad) and Root Mean Square Propagation (RMSProp).
    opt = torch.optim.Adam(params=LR.parameters() , lr=0.001)
    LR.train()
    n=30
    for epoch in range(n):
        #opt.zero_grad() clears old gradients, otherwise they would be added to the new gradients during the backward pass. 
        opt.zero_grad()
        predict = LR(text_train)
        #A loss function, also known as a cost function or objective function, is a crucial component in machine learning and neural networks. 
        #It measures how well a model's predictions match the actual target values. 
        #The goal of training a model is to minimize the loss function, thereby improving the model's performance.
        loss = criterion(predict, spam_train)
        pred = torch.max(predict, 1)[1].eq(spam_train).sum()
        acc = pred * 100.0 / len(text_train)
        print('Epoch:',epoch+1)
        print('loss:',loss.item())
        print('Accuracy:', acc.numpy())
        loss.backward()
        opt.step()
    return LR,criterion
LR,criterion=LR(text_train,spam_train)

Epoch: 1
loss: 0.6833588480949402
Accuracy: 74.41806
Epoch: 2
loss: 0.6792413592338562
Accuracy: 76.16387
Epoch: 3
loss: 0.6586338877677917
Accuracy: 76.18715
Epoch: 4
loss: 0.6273149847984314
Accuracy: 76.6527
Epoch: 5
loss: 0.5907794833183289
Accuracy: 76.6527
Epoch: 6
loss: 0.5509103536605835
Accuracy: 76.6527
Epoch: 7
loss: 0.5087113380432129
Accuracy: 76.6527
Epoch: 8
loss: 0.46522727608680725
Accuracy: 76.6527
Epoch: 9
loss: 0.421738862991333
Accuracy: 76.67598
Epoch: 10
loss: 0.37988772988319397
Accuracy: 77.397575
Epoch: 11
loss: 0.3416287899017334
Accuracy: 78.53818
Epoch: 12
loss: 0.3078572452068329
Accuracy: 79.88827
Epoch: 13
loss: 0.2782336175441742
Accuracy: 81.843575
Epoch: 14
loss: 0.2505125403404236
Accuracy: 88.31471
Epoch: 15
loss: 0.22216860949993134
Accuracy: 91.946
Epoch: 16
loss: 0.1944727748632431
Accuracy: 94.87896
Epoch: 17
loss: 0.1681463122367859
Accuracy: 96.857544
Epoch: 18
loss: 0.14405177533626556
Accuracy: 97.71881
Epoch: 19
loss: 0.12285838276147842
Ac

In [9]:
LR.eval()
with torch.no_grad():
    predict = LR(text_test)
    loss = criterion(predict, spam_test)
    pred = torch.max(predict, 1)[1].eq(spam_test).sum()
    print("LR:")
    print ("Accuracy=",format(pred/len(text_test)))

LR:
Accuracy= 0.9916201233863831


In [10]:
def LR_Regularized(text_train,spam_train):
    LR_R = LogisticRegression()
    criterion = nn.CrossEntropyLoss()
    #weight_decay is an additional parameter that adds a penalty to the loss function. 
    #This technique is also known as L2 regularization.
    opt = torch.optim.Adam(params=LR_R.parameters() , lr=0.001, weight_decay=0.001)
    LR_R.train()
    n=30
    for epoch in range(n):
        opt.zero_grad()
        predict = LR_R(text_train)
        loss = criterion(predict, spam_train)
        pred = torch.max(predict, 1)[1].eq(spam_train).sum()
        acc = pred * 100.0 / len(text_train)
        print('Epoch:',epoch+1)
        print('loss:',loss.item())
        print('Accuracy:', acc.numpy())
        loss.backward()
        opt.step()
    return LR_R,criterion
LR_R,criterion=LR_Regularized(text_train,spam_train)

Epoch: 1
loss: 0.6981726884841919
Accuracy: 26.303537
Epoch: 2
loss: 0.6457263827323914
Accuracy: 77.21136
Epoch: 3
loss: 0.5778239369392395
Accuracy: 76.74581
Epoch: 4
loss: 0.5095574855804443
Accuracy: 77.048416
Epoch: 5
loss: 0.4467583894729614
Accuracy: 78.39851
Epoch: 6
loss: 0.38983824849128723
Accuracy: 80.074486
Epoch: 7
loss: 0.3381441533565521
Accuracy: 81.86685
Epoch: 8
loss: 0.29411986470222473
Accuracy: 84.939476
Epoch: 9
loss: 0.2609201669692993
Accuracy: 88.91992
Epoch: 10
loss: 0.23594848811626434
Accuracy: 92.03911
Epoch: 11
loss: 0.21554818749427795
Accuracy: 94.925514
Epoch: 12
loss: 0.1976642906665802
Accuracy: 96.32216
Epoch: 13
loss: 0.18088506162166595
Accuracy: 97.2067
Epoch: 14
loss: 0.16462472081184387
Accuracy: 97.69553
Epoch: 15
loss: 0.14852389693260193
Accuracy: 97.97486
Epoch: 16
loss: 0.13251203298568726
Accuracy: 98.1378
Epoch: 17
loss: 0.11693057417869568
Accuracy: 98.48696
Epoch: 18
loss: 0.10200953483581543
Accuracy: 98.74302
Epoch: 19
loss: 0.087895

In [11]:
LR_R.eval()
with torch.no_grad():
    predict = LR_R(text_test)
    loss = criterion(predict, spam_test)
    pred = torch.max(predict, 1)[1].eq(spam_test).sum()
    print("LR with regularization:")
    print ("Accuracy=",format(pred/len(text_test)))

LR with regularization:
Accuracy= 0.9881284832954407


In [12]:
'''
If we don't do regularization, the accuracy will be around 99.2%
If we use regularization, the accuracy will be around 98.8%
'''

"\nIf we don't do regularization, the accuracy will be around 99.2%\nIf we use regularization, the accuracy will be around 98.8%\n"

In [13]:
# Part 3: Decision Tree
def DT(text_train,spam_train,text_test,md):
    from sklearn import tree
    clf = tree.DecisionTreeClassifier(max_depth=md)
    clf = clf.fit(text_train, spam_train)
    print(clf.get_depth())
    return clf.predict(text_test)

In [14]:
for dep in range(10,20):
    predict = DT(text_train,spam_train,text_test,dep)
    n=0
    for i in range(len(predict)):
        n+=(predict[i] == spam_test[i])
    print("Depth=",dep)
    print ("Accuracy: ",float(n/len(predict)))

10
Depth= 10
Accuracy:  0.9601955413818359
11
Depth= 11
Accuracy:  0.9601955413818359
12
Depth= 12
Accuracy:  0.9629888534545898
13
Depth= 13
Accuracy:  0.9594972133636475
14
Depth= 14
Accuracy:  0.9601955413818359
15
Depth= 15
Accuracy:  0.9594972133636475
16
Depth= 16
Accuracy:  0.9636871218681335
17
Depth= 17
Accuracy:  0.9629888534545898
18
Depth= 18
Accuracy:  0.9622905254364014
19
Depth= 19
Accuracy:  0.9622905254364014


In [15]:
predict = DT(text_train,spam_train,text_test,1000)
n=0
for i in range(len(predict)):
    n+=(predict[i] == spam_test[i])
print ("Accuracy: ",float(n/len(predict)))

63
Accuracy:  0.9685754179954529


In [16]:
'''
We start from maximum depth=1, the accuracy is around 76%, which is pretty low.
When the maximum depth increases, the accuracy gets higher.
The optimum depth is around 70.
However, the accuracy saturates around 96% as maximum depth growth.
When we choose maximum depth as 20, the accuracy is around 96%.
For the sake of complexity, I believe setting maximum depth to 20 is sufficient for a great accuracy.
'''

'\nWe start from maximum depth=1, the accuracy is around 76%, which is pretty low.\nWhen the maximum depth increases, the accuracy gets higher.\nThe optimum depth is around 70.\nHowever, the accuracy saturates around 96% as maximum depth growth.\nWhen we choose maximum depth as 20, the accuracy is around 96%.\nFor the sake of complexity, I believe setting maximum depth to 20 is sufficient for a great accuracy.\n'

In [17]:
# Part 4: random forest
def random_forest(text_train, spam_train, text_test,md):
    from sklearn.ensemble import RandomForestClassifier
    clf = RandomForestClassifier(max_depth=md, random_state=0)
    clf.fit(text_train, spam_train)
    return clf.predict(text_test)

In [18]:
for dep in range(40,50):
    predict = random_forest(text_train, spam_train, text_test,dep)
    n=0
    for i in range(len(predict)):
        n+=(predict[i] == spam_test[i])
    print("Depth=",dep)
    print ("Accuracy: ",float(n/len(predict)))

Depth= 40
Accuracy:  0.9783519506454468
Depth= 41
Accuracy:  0.9790502786636353
Depth= 42
Accuracy:  0.9797486066818237
Depth= 43
Accuracy:  0.9790502786636353
Depth= 44
Accuracy:  0.9818435907363892
Depth= 45
Accuracy:  0.9783519506454468
Depth= 46
Accuracy:  0.9804469347000122
Depth= 47
Accuracy:  0.9846368432044983
Depth= 48
Accuracy:  0.9846368432044983
Depth= 49
Accuracy:  0.9818435907363892


In [19]:
predict = random_forest(text_train, spam_train, text_test,1000)
n=0
for i in range(len(predict)):
    n+=(predict[i] == spam_test[i])
print ("Accuracy: ",float(n/len(predict)))

Accuracy:  0.9860334992408752


In [20]:
'''
We start from maximum depth=1, the accuracy is around 76%, which is pretty low.
When the maximum depth increases, the accuracy gets higher.
However, the accuracy saturates around 98.8% as maximum depth growth.
When we choose maximum depth as 40, the accuracy is around 98%.
For the sake of complexity, I believe set maximum depth to 40 is sufficient for a great accuracy.
It is obvious that random forest is a better model than decision tree because it is the majority vote of decision trees.
The accuracy is 98% vs 96%.
'''

'\nWe start from maximum depth=1, the accuracy is around 76%, which is pretty low.\nWhen the maximum depth increases, the accuracy gets higher.\nHowever, the accuracy saturates around 98.8% as maximum depth growth.\nWhen we choose maximum depth as 40, the accuracy is around 98%.\nFor the sake of complexity, I believe set maximum depth to 40 is sufficient for a great accuracy.\nIt is obvious that random forest is a better model than decision tree because it is the majority vote of decision trees.\nThe accuracy is 98% vs 96%.\n'

In [21]:
# Part 5: Overall
'''
Logistic regression accuracy: 0.9923184514045715
Logistic regression with regularization accuracy: 0.9888268113136292
Decision Tree accuracy: 0.9622905254364014
Random Forest accuracy: 0.9853351712226868
Accuracy rank: Logistic regression>Random Forest>Decision Tree
The logistic regression is slightly better than the random forest model.
The difference is small so maybe when there is a different train/test set, the result may be different.
'''

'\nLogistic regression accuracy: 0.9923184514045715\nLogistic regression with regularization accuracy: 0.9888268113136292\nDecision Tree accuracy: 0.9622905254364014\nRandom Forest accuracy: 0.9853351712226868\nAccuracy rank: Logistic regression>Random Forest>Decision Tree\nThe logistic regression is slightly better than the random forest model.\nThe difference is small so maybe when there is a different train/test set, the result may be different.\n'